In [ ]:
from parameterize import *
system = PrepareQMMM(init_pdb = "test_system.pdb", 
                     distance = 6.0, 
                     num_residues = 2, 
                     guest_resname = "BEN")
qm_guest = PrepareGaussianGuest(charge = 1, 
                                multiplicity = 1)
qm_system = PrepareGaussianHostGuest(charge = 1, 
                                     multiplicity = 1)
params_guest = ParameterizeGuest(vibrational_scaling = 1.00)
qm_host = PrepareGaussianHost(charge = 0, 
                              multiplicity = 1)
params_host = ParameterizeHost(vibrational_scaling = 1.00)
system_guest = GuestAmberXMLAmber(charge = 1, 
                                  num_charge_atoms = 1, 
                                  index_charge_atom_1 = 9, 
                                  charge_atom_1 = 1)
system_host = HostAmberXMLAmber()
guest_openmm = RunOpenMMSims(system_prmtop = "guest_params.prmtop", 
                             system_inpcrd = "guest_params.inpcrd", 
                             system_pdb = "guest_init_II.pdb")
host_openmm = RunOpenMMSims(system_prmtop = "host_params.prmtop", 
                            system_inpcrd = "host_params.inpcrd", 
                            system_pdb = "host.pdb")
merge_host_guest = MergeHostGuestTopology(host_prmtop = "host_params.prmtop", 
                                          guest_prmtop = "guest_params.prmtop", 
                                          host_inpcrd = "host_params.inpcrd",
                                          guest_inpcrd = "guest_params.inpcrd", 
                                          system_prmtop = "system_params.prmtop", 
                                          system_inpcrd = "system_params.inpcrd")
system_openmm = RunOpenMMSims(system_prmtop = "system_params.prmtop", 
                              system_inpcrd = "system_params.inpcrd", 
                              system_pdb = "system.pdb")
guest_torsion_params = TorsionDriveSims(charge = 1, 
                                        multiplicity = 1)
guest_write_params = TorsionDriveParams(num_charge_atoms = 1, 
                                        index_charge_atom_1 = 9, 
                                        charge_atom_1 = 1)

## Initial files needed : Host-Guest PDB

## Step I : Defining QM and MM Regions

In [ ]:
system.clean_up()
system.create_host_guest()
system.realign_guest()
system.get_guest_coord()
system.get_qm_resids()
system.get_host_qm_mm_atoms()
system.save_host_pdbs()
system.get_host_mm_region_atoms()
system.save_host_mm_regions_pdbs()
system.get_qm_mm_regions()

## Step II : Guest QM Calculation 

In [ ]:
qm_guest.write_input()
#qm_guest.run_gaussian()
#qm_guest.get_fchk()

## Step III : QM Calculation of the Host-Guest region for QM - Derived Charges

In [ ]:
qm_system.write_input()
#qm_system.run_gaussian()
#qm_system.get_fchk()
qm_system.get_qm_host_guest_charges()

## Step IV : Guest Re - Parameterization

In [ ]:
params_guest.get_xyz()
params_guest.get_unprocessed_hessian()
params_guest.get_bond_angles()
params_guest.get_hessian()
params_guest.get_atom_names()
params_guest.get_bond_angle_params()
params_guest.get_charges()
params_guest.get_proper_dihedrals()

## Step V : Host (Residues / Region Surrounding the Guest) QM Calculation 

In [ ]:
qm_host.write_input()
#qm_host.run_gaussian()
#qm_host.get_fchk()

## Step VI : Host (Residues / Region Surrounding the Guest)  Re - Parameterization

In [ ]:
params_host.get_xyz()
params_host.get_unprocessed_hessian()
params_host.get_bond_angles()
params_host.get_hessian()
params_host.get_atom_names()
params_host.get_bond_angle_params()
params_host.get_charges()

## Step VII : Generation of Re - Parameterized .prmtop & .inpcrd Files for Guest

In [ ]:
system_guest.generate_xml_from_charged_pdb_sdf()
system_guest.write_system_params()
system_guest.write_reparameterised_system_xml()
system_guest.save_amber_params()
system_guest.analyze_diff_energies()

## Step VIII : Generation of Re - Parameterized .prmtop & .inpcrd Files for the Host

In [ ]:
system_host.serialize_system()
system_host.write_system_params()
system_host.write_reparameterised_system_xml()
system_host.save_amber_params()
system_host.analyze_diff_energies()

## Step IX : OpenMM Test - Run of the Newly Generated Parameter Files for the System

In [ ]:
guest_openmm.run_openmm_prmtop_inpcrd()
guest_openmm.run_openmm_prmtop_pdb()
host_openmm.run_openmm_prmtop_inpcrd()
host_openmm.run_openmm_prmtop_pdb()
merge_host_guest.merge_topology_files()
system_openmm.run_openmm_prmtop_inpcrd()
system_openmm.run_openmm_prmtop_pdb()

### Step X : Re - Parameterization of the torsion angles for the Guest

In [ ]:
guest_torsion_params.write_torsion_drive_run_file()
guest_torsion_params.write_tor_params_txt()
guest_torsion_params.write_psi4_input()
guest_torsion_params.create_non_H_bonded_torsion_drive_dir()
guest_torsion_params.run_torsion_sim()

In [ ]:
guest_write_params.write_reparams_torsion_lines_charged()

In [ ]:
guest_write_params.write_torsional_reparams()

### Step XII : Solvation and Ion addition to the system